# 0. 小組成員

106024518 張家豪  
105024701 李漢岳  
106024521 黃信恩  

# 1. Data Preprocessing

本節，我們主要以下列流程來做資料前處理:  
1. 讀取檔案
2. 擴充斷詞庫
3. 斷詞
4. 建立詞頻字典
5. 篩選重要詞彙
6. 將詞彙轉成向量 (word2vec)

## 1.0 讀取檔案

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import jieba
import operator
import random
from gensim.models import Word2Vec
from sklearn.metrics import accuracy_score

C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## 1.1 擴充斷詞庫

為了更正確的斷詞，我們加入新的斷詞字典 "my.dict.txt" 

In [2]:
jieba.set_dictionary('big5_dict.txt')
jieba.load_userdict('my.dict.txt')

Building prefix dict from C:\Users\User\python\big5_dict.txt ...
Dumping model to file cache C:\Users\User\AppData\Local\Temp\jieba.u52871cbd0654897ca0fc2fbeb8323d83.cache
Loading model cost 0.833 seconds.
Prefix dict has been built succesfully.


## 1.2 斷詞

以下，我們使用jieba加上自創的stop word，對文件進行斷詞  

In [3]:
# 停用詞
with open('stop_words.txt', 'r', encoding='utf8') as f:  
    stops = f.read().split('\n') 
stops.append("\n")
stops.append(" ")
stops.append('，')

# 載入 programs
programs = []
for i in range(1, 9):
    program = pd.read_csv('Program0%d.csv' % (i))
    programs.append(program)
# 載入 questions
questions = pd.read_csv('Question.csv')


# 定義切詞函數
def jieba_lines(lines):
    cut_lines = []
    for line in lines:
        cut_line = jieba.lcut(line)
        qw = [] # 找非停用詞的字
        for qq in cut_line:
            if qq not in stops:
                qw.append(qq)
        cut_lines.append(qw)
    return cut_lines

cut_programs = []
for program in programs:
    episodes = len(program)
    cut_program = []
    
    for episode in range(episodes):
        lines = program.loc[episode]['Content'].split('\n')
        cut_program.append(jieba_lines(lines))
    cut_programs.append(cut_program)


cut_questions = []
n = len(questions)

for i in range(n):
    cut_question = []
    lines = questions.loc[i]['Question'].split('\n')
    cut_question.append(jieba_lines(lines))
    
    for j in range(6):
        line = questions.loc[i]['Option%d' % (j)]
        cut_question.append(jieba.lcut(line))
    
    cut_questions.append(cut_question)
    

In [6]:
#結果檢查
cut_question[0]

[['警告', '不要', '接到', '學校', '打', '來電話'], []]

## 1.3 建立詞頻字典

以下，我們計算各個單詞的出現頻率，以建立詞頻字典。  
這樣做的目的，是為了將頻率過高與過低的詞刪除，以確保斷出來之詞彙具有一定重樣性  

In [9]:
# 建立字典
def add_word_dict(w):
    if not w in word_dict:
        word_dict[w] = 1
    else:
        word_dict[w] += 1


word_dict = dict()

for program in cut_programs:
    for lines in program:
        for line in lines:
            for w in line:
                add_word_dict(w)

for question in cut_questions:
    lines = question[0]
    for line in lines:
        for w in line:
            add_word_dict(w)
    
    for i in range(1, 7):
        line = question[i]
        for w in line:
            add_word_dict(w)

word_dict = sorted(word_dict.items(), key=operator.itemgetter(1), reverse=True)

# 抽取一定數量的詞頻字典
VOC_SIZE = 130000
VOC_START = 20

voc_dict = word_dict[VOC_START:VOC_START+VOC_SIZE]

# 設定 辭典列表
voc_dict2 = []
for i in range(len(voc_dict)):
    voc_dict2.append(voc_dict[i][0] )

## 1.4 統整所有training set與testing set的句子

由於後續我們將以word2vec的方式建立單詞的向量，以衡量詞與詞間的距離，  
因此，我們在此階段，將統整手邊所有句子成為句庫，以進行單詞轉向量的training。  

In [10]:
## 定義:一句話中取只有在字典裡的字
def choosevoc(dat):
    example_doc2 = []
    for w in dat:
        choose = ''
        if w in voc_dict2:
            choose = w
            example_doc2.append(choose)
        else:
            choose = ''
    return example_doc2

# 定義:把每句對話串在一起
def abc(dat):
    doc = []
    for li in dat:
        for w in li:
            doc.append(w)
    return doc 


# 定義:把每句字詞串在一起
def abcd(dat):
    doc = []
    for li in dat:
        for w in li:
            for qq in w:
                doc.append(qq)
    return doc

## 問題與六個選項合併
qaaaaaa=[]
for i in range(len(cut_questions)):
    qaaaaaa.append(abc(cut_questions[i][0]))
    qaaaaaa.append(cut_questions[i][1])
    qaaaaaa.append(cut_questions[i][2])
    qaaaaaa.append(cut_questions[i][3])
    qaaaaaa.append(cut_questions[i][4])
    qaaaaaa.append(cut_questions[i][5])
    qaaaaaa.append(cut_questions[i][6])
print(qaaaaaa[:5])


## 題目跟每個問題合併
def q_combind(question_):
    qa = []
    for i in range(len(cut_questions)):
        c = []
        c.append(abc(cut_questions[i][0]))
        c.append(cut_questions[i][question_])
        qa.append(c)
    return(qa)

qa1 = q_combind(1)
qa2 = q_combind(2)
qa3 = q_combind(3)
qa4 = q_combind(4)
qa5 = q_combind(5)
qa6 = q_combind(6)

[['送', '錢包', '看', '是不是', '這個', '就是', '這個', '哪裡', '找到', '它'], ['你', '看', ' ', '這是', '我', '新', '買', '的', '錢包'], ['我', '的', '錢包', '不見了', '啦'], ['以後', '上', '網咖', '的', '錢包', '在', '我', '身上'], ['什麼', '有錢', '包場']]


## 1.5 篩選重要詞彙

In [ ]:
## 取 qaX 中有在字典裡的字為 new_qaX
## 定義:取 qaX 中有在字典裡的字為 new_qaX
def indic(data):
    new_data = data
    for i in range(len(data)):
        for j in range(2):
            new_data[i][j] = choosevoc(data[i][j])
    return(new_data)

new_qa1 = indic(qa1)
new_qa2 = indic(qa2)
new_qa3 = indic(qa3)
new_qa4 = indic(qa4)
new_qa5 = indic(qa5)
new_qa6 = indic(qa6)

# 1.6 將詞彙轉向量 (word2vec)

In [ ]:
train_data = abcd(cut_programs)+qaaaaaa

size_ = 500
model = Word2Vec(train_data, size=size_, window=9, min_count=1, workers=4, sg=1, iter=18, hs=1,seed=123)

# 2. Classifier Building

模型建立的部分，我們採取兩種方式來做預測  
1. 計算問題句與回答句間，詞與詞的所有距離，並取前k小的做平均  
2. 將問題句與回答句內的所有詞向量先各自加總，再計算兩個加總後向量的距離  

以下呈現建模過程與結果:  

## 2.1 詞與詞距離

In [ ]:
def qadist(data,k=2,weight= None,dist =1):
    result = []
    for t in range(len(data)):
        sim = []
        for i in range(len(data[t][0])):
            for j in range(len(data[t][1])):
                if dist == 1:
                    distance = -1*model.wv.similarity(data[t][0][i],data[t][1][j])
                elif dist ==2:
                    distance = scipy.spatial.distance.cosine(model.wv[data[t][0][i]],
                                                             model.wv[data[t][1][j]])
                else:
                    distance = scipy.spatial.distance.correlation(model.wv[data[t][0][i]],
                                                                  model.wv[data[t][1][j]])
                
                sim.append(distance)

        sim = np.mean(sorted(sim)[:k])
        result.append(sim)
    return(result)

In [ ]:
import scipy

result = {}
for k_idx in range(5,16):
    for method in range(1,4):
        qacc = np.column_stack((qadist(new_qa1,k=k_idx,dist=method),
                        qadist(new_qa2,k=k_idx,dist=method),
                        qadist(new_qa3,k=k_idx,dist=method),
                        qadist(new_qa4,k=k_idx,dist=method),
                        qadist(new_qa5,k=k_idx,dist=method),
                        qadist(new_qa6,k=k_idx,dist=method)))
        result[''+str(k_idx)+str(method)]
        print(k_idx, method, )

In [ ]:
qacc_label = qacc.argmin(axis=1)
output4 = {'Id':range(0,500),'Answer':qacc_label}
output4 = pd.DataFrame(output4,columns=['Id','Answer'])
print(output4[:5])
output4.to_csv('output_wulala.csv',header=True,index=False)

## 2.2 句與句的距離

把每個詞相加後，找出兩個向量的cos距離。

這邊有試過將向量做加權獲取平均，在public LB上表現沒有直接相加好。

In [ ]:
# 相似度
def qcc_(data1,data2,method):
    data_combined = []
    for i in range(1):
        if data1==[]: # 問題
            first = np.zeros(size_,)
        else: 
            first = model.wv[data1].sum(axis=0)
            
        if data2==[]: # 答案
            second = np.zeros(size_,)
        else: 
            second = model.wv[data2].sum(axis=0)
        # 定義距離
        if data1==[] or data2==[]: 
            dist = 0
        else:
            if method==1:
                dist = scipy.spatial.distance.cosine(first, second) 
            elif method==2:
                dist = scipy.spatial.distance.correlation(first, second)
            else:
                dist = np.corrcoef(first,second)[0,1] 
        data_combined.append(dist)
    return(data_combined)

In [ ]:
odel = Word2Vec(mylist, size=250, window=9, min_count=1, 
                workers=4, sg=1, iter=18, hs=1,seed=123)
                
qacc = np.column_stack((qcc(new_qa1,1),qcc(new_qa2,1),qcc(new_qa3,1),
                        qcc(new_qa4,1),qcc(new_qa5,1),qcc(new_qa6,1))) # cos
qacc_label = qacc.argmin(axis=1)
output4 = {'Id':range(0,500),'Answer':qacc_label}
output4 = pd.DataFrame(output4,columns=['Id','Answer'])
output4.to_csv('output_250_9_18.csv',header=True,index=False)

# 3. Conclusion

## 3.1 方法比較結果

我們嘗試過以下方法  
1. w2v + random forest  
2. w2v + xgboost
3. Doc2vec
4. 單純比較詞與詞間的距離

我們最後發現，反而是最單純的方法四，效果最好。  
在這個方法中，我們嘗試了兩種計算距離的方法。  
第一種方法是我們將問題句裡面的所有詞，與回答句的所有詞作兩兩組合，計算出所有的combination的距離後，取前k小的距離做平均。  
第二種方法是我們先各自加總問題句裡面所有的詞的向量，以及回答句裡面所有詞的向量，再計算此兩項量的距離。  
而距離的定義方式採取 -similarity,  scipy.spatial.distance.cosine, 與scipy.spatial.distance.correlation三種方式。  

結果發現，成效最為明顯的是各自加總在計算距離的方法。於testing set上的準確率為 0.66。  
而另一種方法的準確率較差，於testing set上的準確率為0.56。  

## 3.2 缺點與未來改進方向

我們覺得這次的比賽，應該是犯了模型不穩定的錯誤。  
因為，在比賽期間我們選出最好的model，在比賽結束後的表現反而較差。  
我們曾思考過是否為over-fitting的現象，但其實我們並沒有真的使用任何machine learing的方法，也沒有適配過於複雜的函數。  
相反的，我們只有使用word2vec的方式去計算詞與詞的距離。  
因此，也許這種過於簡單的方式並不robust，我們無法只根據比賽期間的準確率來選擇模型。  

未來在進行相似的分析時，我們也許可以考慮以下兩種方式來改進:  
1. 在目前的架構下做改進:
2. 改使用那天分享那些隊的方式

對於第一點，我們可以再多建幾個新feature，使用較不複雜的model(如logistic regression)，並對training set做cross-validation來選擇模型，而不是像這次比賽時，我們只以比賽期間的提交數據來選擇模型。如此一來，應該可以加強選模的穩定性。  

對於第二點，我們可以嘗試bi-LSTM的方法以及切字上採用character去切來提升我們的模型成效。